In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(777)

print(tf.__version__)
print(np.__version__)

2.3.0
1.18.5


In [3]:
#IMDB에서 50000의 영화 리뷰를 가지고 온다
#10000개의 빈도수가 높은 단어를 학습시 vector에 사용

imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0])

17465344/17464789 [==============================] - 5s 0us/step
Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 447

In [5]:
#word를 integer index로 구분
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}
#공백
word_index["<PAD>"] = 0
#시작값
word_index["<START>"] = 1
#모르는 단어일 경우
word_index["<UNK>"] = 2
#사용하지 않는 단어
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[4])

"<START> worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the <UNK> and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life"

In [7]:
#data를 기준으로 분류 모델 만들기
#길이는 pad값을 주어 256으로 맞춰줌
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value = word_index["<PAD>"],
                                                       padding = 'post',
                                                       maxlen = 256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value = word_index["<PAD>"],
                                                       padding = 'post',
                                                       maxlen = 256)

print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

In [10]:
#model에 대한 정의
#입력 size와 학습시킬 layer의 크기, activation function 정의

#입력 size
vocab_size = 10000

#model 정의
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation = tf.nn.relu))
model.add(keras.layers.Dense(1, activation = tf.nn.sigmoid))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [11]:
#adam optimizer, CE loss 선언
model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [12]:
#모델을 평가할 test data 정의
#10000을 기준으로 학습

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs = 40,
                   batch_size = 512,
                   validation_data = (x_val, y_val),
                   verbose = 1)

Epoch 1/40
30/30 [==============================] - 2s 50ms/step - loss: 0.6916 - accuracy: 0.5673 - val_loss: 0.6895 - val_accuracy: 0.6564
Epoch 2/40
30/30 [==============================] - 2s 51ms/step - loss: 0.6852 - accuracy: 0.7347 - val_loss: 0.6806 - val_accuracy: 0.7087
Epoch 3/40
30/30 [==============================] - 1s 48ms/step - loss: 0.6720 - accuracy: 0.7507 - val_loss: 0.6642 - val_accuracy: 0.7210
Epoch 4/40
30/30 [==============================] - 2s 53ms/step - loss: 0.6492 - accuracy: 0.7607 - val_loss: 0.6381 - val_accuracy: 0.7668
Epoch 5/40
30/30 [==============================] - 2s 57ms/step - loss: 0.6153 - accuracy: 0.7868 - val_loss: 0.6018 - val_accuracy: 0.7712
Epoch 6/40
30/30 [==============================] - 1s 45ms/step - loss: 0.5721 - accuracy: 0.8097 - val_loss: 0.5605 - val_accuracy: 0.8040
Epoch 7/40
30/30 [==============================] - 1s 36ms/step - loss: 0.5241 - accuracy: 0.8295 - val_loss: 0.5163 - val_accuracy: 0.8192
Epoch 8/40
30

In [14]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 3s 4ms/step - loss: 0.3307 - accuracy: 0.8727: 0s - loss: 0.3280 - ac
[0.3306798040866852, 0.8726800084114075]
